# Predict

This notebook predicts which songs you will like given a playlist, using any classifier saved by the **Train** notebook.

In [ ]:
import sklearn
from pkg.config import CLIENT_ID, CLIENT_USERNAME, CLIENT_SECRET
from pkg.serialize import load_classifier, load_all_classifiers
from pkg.spotipy import get_playlist_data, login_to_spotify
from pkg.classify import get_playlist_features, classify_playlist

# "log in" to Spotify
SPOTIPY_OBJECT = login_to_spotify(CLIENT_USERNAME, CLIENT_ID, CLIENT_SECRET)

### Specify Classifier

Run the following cell, and enter the full filename of any classifier file in the 'classifiers' folder, e.g. user_name2019-04-21@164523.pkl.

You can specify a single classifier by filename, **or** leave it blank to run all classifiers in the `/classifiers` directory.

In [ ]:
print('Please enter a classifier filename or leave blank, and press enter:')
CLASSIFIER_FILENAME = input()
# Use the full name of any file in the 'classifiers' folder, e.g. user_name2019-04-21@164523.pkl
# Provide a blank input to run all classifiers.

if CLASSIFIER_FILENAME:
    CLASSIFIERS = [(load_classifier(CLASSIFIER_FILENAME), CLASSIFIER_FILENAME)]
else:
    CLASSIFIERS = load_all_classifiers()
    
print('\nClassifier(s) successfully loaded.')

### Specify Playlist

Run the following cell, and enter the full playlist ID (Spotify URI) of the playlist you would like to run the classifier on. Your playlist ID is accessible by right-clicking on a playlist within Spotify -> Share -> Copy Spotify URI, and removing the `spotify:user:USERNAME:playlist:` portion of the Spotify URI.

e.g. `spotify:user:USERNAME:playlist:PLAYLIST_ID` (Spotify URI)

In [ ]:
print('Please enter a Spotify playlist ID or URI, and press enter:')
PLAYLIST = input()
# You must specify a playlist
# e.g. spotify:user:USERNAME:playlist:PLAYLIST_ID (Spotify URI)

if not PLAYLIST: raise RuntimeError('Please specify a playlist.')

# download Spotify playlist data
playlist_data, playlist_name = get_playlist_data(
    SPOTIPY_OBJECT,
    CLIENT_ID,
    PLAYLIST)

print('\nPlaylist successfully downloaded.')

# parse/wrangle playlist data
playlist_feature_data, playlist_data_dict = get_playlist_features(
    SPOTIPY_OBJECT,
    playlist_data,
    playlist_name
)

print('Playlist successfully parsed.')

### Predict

Run the below cell to get your recommendations for the current playlist and classifier(s).

In [ ]:
for clf in CLASSIFIERS:
    classify_playlist(clf, playlist_feature_data, playlist_data_dict)